In [ ]:
import numpy
from matplotlib import pyplot as plot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import to_categorical

In [ ]:
from keras.datasets import cifar10
(train_data_raw, train_labels_raw), (test_data_raw, test_labels_raw) = cifar10.load_data()

In [ ]:
print("Training data shape: ", train_data_raw.shape, train_labels_raw.shape)
print("Testing data shape: ", test_data_raw.shape, test_labels_raw.shape)

In [ ]:
plot.figure(figsize=[4, 2])

plot.subplot(121)
plot.imshow(train_data_raw[0,:,:], cmap='gray')
plot.title=("Ground Truth: {}".format(train_labels_raw[0]))

plot.subplot(122)
plot.imshow(test_data_raw[0,:,:], cmap='gray')
plot.title=("Ground Truth: {}".format(test_labels_raw[0]))

In [ ]:
classes = numpy.unique(train_labels_raw)
print("Classes: ", classes)

In [ ]:
n_classes = len(classes)
n_rows, n_cols, n_dimens = train_data_raw.shape[1:]
input_shape = (n_rows, n_cols, n_dimens)

In [ ]:
train_data = train_data_raw.reshape(train_data_raw.shape[0], n_rows, n_cols, n_dimens)
test_data = test_data_raw.reshape(test_data_raw.shape[0], n_rows, n_cols, n_dimens)

train_data = train_data.astype('float32') #unit8 -> float32
test_data = test_data.astype('float32')

In [ ]:
train_data /= 255
test_data /= 255

In [ ]:
train_labels = to_categorical(train_labels_raw)
test_labels = to_categorical(test_labels_raw)

In [ ]:
batch_size = 256
epochs = 50

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))
model.summary()

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
import pickle

file = open("model.pickle", "wb")
pickle.dump(model, file)
file.close()


file = open("history.pickle", "wb")
pickle.dump(history, file)
file.close()

In [ ]:
import pickle

file = open("model.pickle", "rb")
model = pickle.load(file)
file.close()

file = open("history.pickle", "rb")
history = pickle.load(file)
file.close()

In [ ]:
plot.figure(figsize=[8,6])
plot.plot(history.history['acc'], 'r', linewidth=3.0)
plot.plot(history.history['val_acc'], 'b', linewidth=3.0)
plot.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18.0)
plot.xlabel('Epochs', fontsize=16.0)
plot.ylabel('Accuracy', fontsize=16)
plot.title('Accuracy Curves', fontsize=16.0)
plot.show()

In [ ]:
plot.figure(figsize=[8,6])
plot.plot(history.history['loss'], 'r', linewidth=3.0)
plot.plot(history.history['val_loss'], 'b', linewidth=3.0)
plot.legend(['Training Loss', 'Validation Loss'], fontsize=18.0)
plot.xlabel('Epochs', fontsize=16.0)
plot.ylabel('Loss', fontsize=16)
plot.title('Loss Curves', fontsize=16.0)
plot.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes, activation='softmax'))

model2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

data_generator = ImageDataGenerator(
    rotation_range = 10.0,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.0,
    zoom_range = 0.1,
    horizontal_flip=True,
    vertical_flip=True)

history2 = model2.fit_generator(
    data_generator.flow(
        train_data, 
        train_labels, 
        batch_size=batch_size
    ), 
    epochs=epochs, 
    validation_data=(test_data, test_labels),
    steps_per_epoch=int(numpy.ceil(train_data.shape[0]/float(batch_size))), 
    workers=4
)

model2.summary()

In [ ]:
model2.evaluate(test_data, test_labels)

In [ ]:
import pickle

file = open("model2.pickle", "wb")
pickle.dump(model2, file)
file.close()

file = open("history2.pickle", "wb")
pickle.dump(history2, file)
file.close()

In [ ]:
import pickle

file = open("model2.pickle", "rb")
model2 = pickle.load(file)
file.close()

file = open("history2.pickle", "rb")
history2 = pickle.load(file)
file.close()

In [ ]:
plot.figure(figsize=[8,6])
plot.plot(history2.history['acc'], 'r', linewidth=3.0)
plot.plot(history2.history['val_acc'], 'b', linewidth=3.0)
plot.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18.0)
plot.xlabel('Epochs', fontsize=16.0)
plot.ylabel('Accuracy', fontsize=16)
plot.title('Accuracy Curves', fontsize=16.0)
plot.show()

In [ ]:
plot.figure(figsize=[8,6])
plot.plot(history2.history['loss'], 'r', linewidth=3.0)
plot.plot(history2.history['val_loss'], 'b', linewidth=3.0)
plot.legend(['Training Loss', 'Validation Loss'], fontsize=18.0)
plot.xlabel('Epochs', fontsize=16.0)
plot.ylabel('Loss', fontsize=16)
plot.title('Loss Curves', fontsize=16.0)
plot.show()

In [ ]:
model.predict(numpy.expand_dims(test_data[0], axis=0))
model.predict_classes(numpy.expand_dims(test_data[0], axis=0))

In [ ]:
test_labels[0]

In [ ]:
from keras.preprocessing import image
import cv2

In [ ]:
img = image.load_img("./images/dog1.jpeg", target_size=(32, 32))
img_data = image.img_to_array(img)

model.predict_classes(numpy.expand_dims(img_data, axis=0))

In [ ]:
import scipy.ndimage

def dodge_merge(front, rear):
    res = front * 255 / (255 - rear)
    res[res > 255] = 255
    res[rear==255] = 255
    return res.astype('uint8')

img = image.load_img("./images/car1.jpeg", target_size=(256, 256))

img_data = image.img_to_array(img)
gray_img = numpy.dot(test_data[0], [0.299, 0.587, 0.114])
inverted_gray_img = 255 - gray_img

blur_image = scipy.ndimage.filters.gaussian_filter(
    inverted_gray_img, 
    sigma=100
)

sketch = dodge_merge(blur_image, gray_img)

plot.imshow(img, cmap='gray')
plot.title=("Ground Truth: {}".format(test_labels_raw[0]))